In [1]:
import pandas as pd
train=pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")
train

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
287108,fffdfb56fdf1a12d364562cc2b9b1d4de7481dee,By . James Rush . Former first daughter Chelse...,Chelsea Clinton said question of running for o...
287109,fffeecb8690b85de8c3faed80adbc7a978f9ae2a,An apologetic Vanilla Ice has given his first ...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,ffff5231e4c71544bc6c97015cdb16c60e42b3f4,America's most lethal sniper claimed he wished...,America's most lethal sniper made comment in i...
287111,ffff924b14a8d82058b6c1c5368ff1113c1632af,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...",A swarm of more than one million has crossed b...


In [2]:
train.isnull().sum()

id            0
article       0
highlights    0
dtype: int64

In [3]:
import tensorflow as tf
tokenizer=tf.keras.preprocessing.text.Tokenizer()
# random_sentence="12th Fail: Did you know IPS officer Manoj Kumar Sharma, IRS officer Shraddha Joshi had cameos in Vikrant Massey film?"
def tokenizenews(random_sentence):
    tokenizer=tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts([random_sentence])
    sequence = tokenizer.texts_to_sequences([random_sentence])
    sequence_length = 50  # Adjust as needed
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=sequence_length)
    return padded_sequence
# train['article']=train['article'].apply(tokenizenews)
# train['highlights']=train['highlights'].apply(tokenizenews)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from tensorflow.keras.models import Model
def encoder(array):
    vocab_size = 10000
    embedding_dim = 200
    encoder_hidden_states=[]
    sequence_length = 50
    input_sequence = tf.keras.Input(shape=(sequence_length, ))
    embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_sequence)
    encoder_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128 , return_sequences=True))(embedding_layer)
    encoded_representation = tf.keras.layers.Concatenate()([encoder_lstm[:, -1, :], encoder_lstm[:, 0, :]])
    model = tf.keras.Model(inputs=input_sequence, outputs=encoded_representation )
    print(array)
    for random_sentence in array:
        result = model.predict(random_sentence)
        encoder_hidden_states.append(result)
    return encoder_hidden_states

In [5]:
def decoder(encoder_hidden_states):
    loss = 0
    context_vector = None
    decoder_state = None
    sequence_length = 50
    hidden_size = 128
    embedding_dim = 200
    v = tf.Variable(tf.random.normal(shape=(hidden_size,)), trainable=True, name='v')
    Wh = tf.Variable(tf.random.normal(shape=(hidden_size, hidden_size)), trainable=True, name='Wh')
    Ws = tf.Variable(tf.random.normal(shape=(hidden_size, embedding_dim)), trainable=True, name='Ws')
    battn = tf.Variable(tf.zeros(shape=(hidden_size,)), trainable=True, name='battn')
    for t in range(len(train['highlights'])):
        input_sequence = tf.keras.Input(shape=(sequence_length, ))
        embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_sequence)
        model = tf.keras.Model(inputs=input_sequence, outputs=embedding_layer)
        output_embeddings = model.predict(padded_sequence)
        attention_scores = []
        for i in range(len(encoder_hidden_states)):
            e_t_i = v * tf.nn.tanh(Wh * encoder_hidden_states[i] + Ws * decoder_state + battn)
            attention_scores.append(e_t_i)
        attention_distributon=tf.nn.softmax(attention_scores)
        for i in range(len(encoder_hidden_states)):
            context_vector += attention_distributon[i] * encoder_hidden_states[i]
        concatenated_input = tf.concat([context_vector,output_embeddings], axis=-1)
        input_sequence1 = tf.keras.Input(shape=(sequence_length, ))
        layer1=tf.keras.layers.Dense(128)(concatenated_input)
        layer2=tf.keras.layers.Dense(1)(layer1)
        model1 = tf.keras.Model(inputs=input_sequence1, outputs=layer2)
        output_embeddings1 = model1.predict(concatenated_input)
        vocab_distribution=tf.nn.softmax(output_embeddings1)
        target_word_prob = vocabulary_distribution[train['highlights'][t]]
        timestep_loss = -tf.math.log(target_word_prob)
        decoder_state=tf.concat([decoder_state,context_vector], axis=-1)
        loss += timestep_loss
        print(loss)

In [6]:
encoder([tokenizenews("Are you a curious mind, wanting to harness the latest developments in AI to transform the world? Or do you have the ability to invent for the future? EnCode is ready to provide you with a fantastic chance! UDGAM 2024, IIT Guwahati proudly presents the third edition of its flagship AI Hackathon: EnCode, presented by DevRev to inspire and support innovators!")])

[array([[14,  5,  1, 15,  2, 16, 17, 18,  3, 19,  2, 20,  1, 21, 22,  2,
        23,  6, 24, 25,  1, 26,  3, 27,  4, 28, 29, 30, 31, 32, 33, 34,
        35,  2, 36, 37, 38, 39, 40,  5, 41,  6, 42, 43, 44,  1, 45, 46,
        47, 48]], dtype=int32)]
1/1 [==============================] - 3s 3s/step


[array([[ 6.11115899e-03,  1.25355115e-02,  5.57622779e-03,
         -7.53099192e-03, -1.97249316e-02, -4.64534014e-03,
         -2.09581660e-04,  1.49360700e-02, -3.01640015e-03,
          9.97523591e-03, -1.07667400e-02, -7.22965458e-04,
         -5.57800010e-03, -1.72020365e-02, -4.96368296e-03,
         -4.21260344e-03,  1.22842635e-03, -2.10164557e-03,
          8.74959491e-03,  1.05933174e-02, -2.25211913e-03,
          1.43973622e-02,  6.04832126e-03,  5.10310614e-03,
          1.13598481e-02,  6.02189032e-03,  5.22831222e-04,
          3.71064269e-03, -3.09418188e-03, -7.17918109e-03,
          6.31276378e-03, -1.47320621e-03,  4.91324347e-03,
          9.64182243e-03, -1.04504451e-02, -6.67039678e-03,
          1.65494848e-02, -7.05306232e-03,  5.93187800e-03,
         -3.33923497e-03, -1.25024961e-02,  7.41695566e-03,
          4.21492429e-03,  6.68431353e-03,  6.90144114e-03,
          4.01959475e-03,  2.60822382e-02,  1.13393273e-03,
          3.27864283e-04, -5.72287245e-0